In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

In [2]:
df = pd.read_csv('v_data.csv')

In [3]:
#This is a pruned dataframe with a subset of the columns that we'll be using for this exploratory anaylsis
values = ['total_vaccinations', 'people_vaccinated', 'daily_vaccinations_per_million']
prn_df = df[['location', 'date', 'total_vaccinations', 'people_vaccinated', 'daily_vaccinations_per_million']]

In [4]:
#This is to see which locations have the most missing data, we could have ran count() on any of the columns 
#since dropna() drops all Na rows
count_series = prn_df.dropna().groupby('location')['date'].count()

In [5]:
#describe tells us the average number of entries is 111, but std-dev is fairly significant at 93, we'll need to look at the list 
#of indices (locations) with above average sampling, as better reporting probably implies higher quality data
count_series.describe()['mean']

111.48497854077253

In [6]:
x = count_series.describe()['mean']
ab_avg = [ind for ind in count_series.index if count_series[ind]>x]
ab_avg

['Africa',
 'Argentina',
 'Aruba',
 'Asia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belgium',
 'Bolivia',
 'Brazil',
 'Bulgaria',
 'Cambodia',
 'Canada',
 'Chile',
 'Colombia',
 "Cote d'Ivoire",
 'Croatia',
 'Curacao',
 'Czechia',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'England',
 'Estonia',
 'Europe',
 'European Union',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Gibraltar',
 'Greece',
 'Guatemala',
 'High income',
 'Hong Kong',
 'Hungary',
 'India',
 'Indonesia',
 'Ireland',
 'Isle of Man',
 'Israel',
 'Italy',
 'Japan',
 'Kazakhstan',
 'Kenya',
 'Latvia',
 'Lebanon',
 'Liechtenstein',
 'Lithuania',
 'Low income',
 'Lower middle income',
 'Luxembourg',
 'Macao',
 'Malawi',
 'Malaysia',
 'Maldives',
 'Malta',
 'Mexico',
 'Moldova',
 'Mongolia',
 'Montenegro',
 'Morocco',
 'Namibia',
 'Nepal',
 'New Zealand',
 'North America',
 'Northern Ireland',
 'Norway',
 'Oceania',
 'Palestine',
 'Panama',
 'Paraguay',
 'Peru',
 'Poland',
 '

In [7]:
#List seems fairly varied, we'll be arbitrarily selecting some countries to include in our analyses
Countries = ['World', 'Asia', 'Africa', 'Europe', 
             'France', 'Italy', 'Spain', 'Germany', 'United Kingdom', 'Russia', 'Scotland', 'Norway', 'Belgium',
             'United States', 'Canada', 'Mexico', 'Brazil', 
             'Zimbabwe', 'South Africa', 'Morocco', 'Algeria', 'Tunisia',
             'Taiwan', 'Hong Kong', 'India', 'Thailand', 'Sri Lanka', 'Japan', 'South Korea', 'Vietnam', 'Indonesia', 'Australia',
             'Qatar', 'Bahrain', 'Israel', 'Turkey',
             'Low income', 'High income', 'Upper middle income', 'Lower middle income' 
            ]
#final_df = prn_df.dropna().drop(index=[prn_df[ind].index for ind in list((set(count_series.index)-set(Countries)))])

In [8]:
#Computing a final dataframe 
final_df = prn_df.pivot_table(values, index=['location','date']).loc[Countries]

In [9]:
final_df.fillna(0, inplace=True)

In [10]:
final_df.reset_index(level=1)

,date,daily_vaccinations_per_million,people_vaccinated,total_vaccinations
location,,,,
World,2020-12-01,0.0,1.0,1.000000e+00
World,2020-12-02,0.0,1.0,1.000000e+00
World,2020-12-03,0.0,1.0,1.000000e+00
World,2020-12-04,0.0,2.0,2.000000e+00
World,2020-12-05,0.0,2.0,2.000000e+00
...,...,...,...,...
Lower middle income,2021-09-10,4202.0,879750877.0,1.237931e+09
Lower middle income,2021-09-11,4464.0,889018072.0,1.253306e+09
Lower middle income,2021-09-12,4600.0,900438065.0,1.273433e+09


In [11]:
#reading the second DF on GDPpc
#path = path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
df2 = pd.read_csv('tgdp.csv')
df2

,Entity,Code,Year,"GDP per capita, PPP (constant 2017 international $)"
0,Afghanistan,AFG,2002,1189.784668
1,Afghanistan,AFG,2003,1235.810063
2,Afghanistan,AFG,2004,1200.278013
3,Afghanistan,AFG,2005,1286.793659
4,Afghanistan,AFG,2006,1315.789117
...,...,...,...,...
7104,Zimbabwe,ZWE,2016,3173.610829
7105,Zimbabwe,ZWE,2017,3274.611198
7106,Zimbabwe,ZWE,2018,3341.665418
7107,Zimbabwe,ZWE,2019,3027.656038


In [12]:
#pd.merge(final_df.reset_index(level=1),df2[df2['Year']>=2019],how='left', left_on='location', right_on='Entity')

In [13]:
gdpdf = pd.merge(final_df.reset_index()['location'], df2[df2['Year']>=2019], how='inner', left_on = 'location', right_on = 'Entity').drop(columns=['Entity'])
gdpdf.drop_duplicates().reset_index(drop=True)

,location,Code,Year,"GDP per capita, PPP (constant 2017 international $)"
0,World,OWID_WRL,2019,16915.287846
1,World,OWID_WRL,2020,16194.040310
2,France,FRA,2019,45834.166912
3,France,FRA,2020,42025.617373
4,Italy,ITA,2019,42662.522370
...,...,...,...,...
62,High income,NaN,2020,47785.704241
63,Upper middle income,NaN,2019,17440.339444
64,Upper middle income,NaN,2020,17145.955042
65,Lower middle income,NaN,2019,7106.967485


In [30]:
#Extracting countries from GDP-DF
countries = final_df.reset_index(level=0)['location', ''].drop_duplicates()
countries.to_csv('../staging/countries.csv', index=False)

environ{'SHELL': '/bin/bash',
        'WSL_DISTRO_NAME': 'Ubuntu-20.04',
        'NAME': 'DESKTOP-G7SJ5I5',
        'PWD': '/home/aziz/airflow/dags/vax-dbrd/scripts',
        'LOGNAME': 'aziz',
        'HOME': '/home/aziz',
        'LANG': 'C.UTF-8',
        'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01;31:*.7z=01;31:*.rz=01;31:*.cab=01;31:*.wim=01;31:*.swm=01;31:*.dwm=01;31:*.esd=01;31:*.jpg=01;35:*.jpeg=0